In [8]:
import torch
from tqdm import tqdm
from torchvision import transforms, models
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms
import os
import random

import numpy as np
from tqdm import tqdm
from PIL import Image
import ssl
import gc  

repo_dir = '/mnt/storage1/Haoran/projects/retina/retina-dinov2'
model_name = 'vitl16'
pretrained_model = False
if pretrained_model:
    train_type = 'pretrained'
else:
    train_type = 'scratch'
feature_save_name = f'{model_name}_{train_type}'


In [9]:
import os
from torchvision import transforms
from torch.utils.data import Dataset
from PIL import Image

def list_files(dataset_path):
    print("Listing files in:", dataset_path)
    images = []
    for root, _, files in sorted(os.walk(dataset_path)):
        for name in sorted(files):
            images.append(os.path.join(root, name))
    print(f"Found {len(images)} files.")
    return images

class CustomImageDataset(Dataset):
    def __init__(self, img_dir):
        self.img_dir = img_dir
        self.images = list_files(self.img_dir)
        self.transform =  transforms.Compose([
                            transforms.Resize(224),
                            transforms.CenterCrop(224),
                            transforms.ToTensor(),
                            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                        ])

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, img_path

    def get_file_names(self):
        file_names = [os.path.basename(path) for path in self.images]
        print(f"Extracted {len(file_names)} file names.")
        return file_names

# Example usage:
dir_path = "/mnt/gvd0n1/Abbas/Projects/Dyer/Danielle/Fundus_batch6_April29/crop/"
dataset = CustomImageDataset(dir_path)
file_names = np.array(dataset.get_file_names())
np.savetxt('/mnt/storage1/Haoran/projects/retina/retina-fundus/data/fundus_image_list.txt', file_names, fmt='%s')


Listing files in: /mnt/gvd0n1/Abbas/Projects/Dyer/Danielle/Fundus_batch6_April29/crop/
Found 21294 files.
Extracted 21294 file names.


In [10]:
from dinov2.models.vision_transformer import vit_base, vit_large

if model_name == 'vitb16':
    model = vit_base(
        patch_size=16,
        img_size=224,
        init_values=1.0,
        block_chunks=0
    )
elif model_name == 'vitl16':
    model = vit_large(
        patch_size=16,
        img_size=224,
        init_values=1.0,
        block_chunks=0
    )

In [11]:
def modify_keys(pretrained_weights):
    new_weights = {}
    for k, v in pretrained_weights.items():
        if k.startswith('backbone.'):
            k = k.replace('backbone.', '')
        parts = k.split('.')
        if len(parts) > 1 and parts[0] == 'blocks':
            k = '.'.join(parts[:1] + parts[2:])

        if parts[0] != 'dino_head':
            new_weights[k] = v

    return new_weights

if pretrained_model == False:
    checkpoint_path = f'/mnt/storage1/Haoran/projects/retina/retina-dinov2/model/{model_name}/eval/training_112499/teacher_checkpoint.pth'
    pretrained_weights = torch.load(checkpoint_path)['teacher']
    pretrained_weights = modify_keys(pretrained_weights)
    ssl._create_default_https_context = ssl._create_unverified_context

    dinov2_model.load_state_dict(pretrained_weights)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print("Using device:", device)
    dinov2_model.to(device)  # Move your model to the GPU
else:
    ssl._create_default_https_context = ssl._create_unverified_context
    dinov2_model = torch.hub.load('facebookresearch/dinov2', f'dinov2_{model_name}')
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print("Using device:", device)
    dinov2_model.to(device)  #

RuntimeError: Error(s) in loading state_dict for DinoVisionTransformer:
	Unexpected key(s) in state_dict: "blocks.12.norm1.weight", "blocks.12.norm1.bias", "blocks.12.attn.qkv.weight", "blocks.12.attn.qkv.bias", "blocks.12.attn.proj.weight", "blocks.12.attn.proj.bias", "blocks.12.ls1.gamma", "blocks.12.norm2.weight", "blocks.12.norm2.bias", "blocks.12.mlp.fc1.weight", "blocks.12.mlp.fc1.bias", "blocks.12.mlp.fc2.weight", "blocks.12.mlp.fc2.bias", "blocks.12.ls2.gamma", "blocks.13.norm1.weight", "blocks.13.norm1.bias", "blocks.13.attn.qkv.weight", "blocks.13.attn.qkv.bias", "blocks.13.attn.proj.weight", "blocks.13.attn.proj.bias", "blocks.13.ls1.gamma", "blocks.13.norm2.weight", "blocks.13.norm2.bias", "blocks.13.mlp.fc1.weight", "blocks.13.mlp.fc1.bias", "blocks.13.mlp.fc2.weight", "blocks.13.mlp.fc2.bias", "blocks.13.ls2.gamma", "blocks.14.norm1.weight", "blocks.14.norm1.bias", "blocks.14.attn.qkv.weight", "blocks.14.attn.qkv.bias", "blocks.14.attn.proj.weight", "blocks.14.attn.proj.b

In [ ]:
dir_path = "/mnt/gvd0n1/Abbas/Projects/Dyer/Danielle/Fundus_batch6_April29/crop/"
dataset = CustomImageDataset(dir_path)

Listing files in: /mnt/gvd0n1/Abbas/Projects/Dyer/Danielle/Fundus_batch6_April29/crop/
Found 21294 files.
Extracted 21294 file names.
['DL6-1-23-3_B-T87L964D_OCT_Image_LE_Line-2023-01-10-09-16-25_fundus_298.tif', 'DL6-1-23-3_B-T87L964D_OCT_Image_LE_Line-2023-01-10-09-16-25_fundus_542.tif', 'DL6-1-23-3_B-T87L964D_OCT_Image_LE_Line-2023-01-10-09-16-25_fundus_786.tif', 'DL6-1-23-3_B-T87L964D_OCT_Image_LE_Line-2023-01-10-09-16-28_fundus_298.tif', 'DL6-1-23-3_B-T87L964D_OCT_Image_LE_Line-2023-01-10-09-16-28_fundus_542.tif', 'DL6-1-23-3_B-T87L964D_OCT_Image_LE_Line-2023-01-10-09-16-28_fundus_786.tif', 'DL6-1-23-3_B-T87L964D_OCT_Image_LE_Line-2023-01-10-09-16-31_fundus_298.tif', 'DL6-1-23-3_B-T87L964D_OCT_Image_LE_Line-2023-01-10-09-16-31_fundus_542.tif', 'DL6-1-23-3_B-T87L964D_OCT_Image_LE_Line-2023-01-10-09-16-31_fundus_786.tif', 'DL6-1-23-3_B-T87L964D_OCT_Image_LE_Line-2023-01-10-09-16-34_fundus_298.tif', 'DL6-1-23-3_B-T87L964D_OCT_Image_LE_Line-2023-01-10-09-16-34_fundus_542.tif', 'DL6-1-

In [ ]:


dir_path = "/mnt/gvd0n1/Abbas/Projects/Dyer/Danielle/Fundus_batch6_April29/crop/"
dataset = CustomImageDataset(dir_path)
print(dataset.__len__())
train_dataloader = DataLoader(dataset, batch_size=64, shuffle=False)

final_img_features = []
final_img_filepaths = []

for image_tensors, file_paths in tqdm(train_dataloader):
    try:
        with torch.no_grad():  # Disable gradient computation
            img_t = image_tensors.to(device)
            image_features = dinov2_model(img_t) #384 small, #768 base, #1024 large
            image_features /= image_features.norm(dim=-1, keepdim=True)
            image_features = image_features.cpu()
            image_features = image_features.tolist()

            # Append data to lists 
            final_img_features.extend(image_features)
            final_img_filepaths.extend(list(file_paths))

        # Explicitly delete tensors to free up memory
        del img_t
        del image_features
        torch.cuda.empty_cache()  # Clear memory cache

    except Exception as e:
        print("Exception occurred: ", e)
        break
    finally:
        # Force garbage collection to run (optional, could be expensive)
        gc.collect()


Listing files in: /mnt/gvd0n1/Abbas/Projects/Dyer/Danielle/Fundus_batch6_April29/crop/
Found 21294 files.
21294


  0%|          | 0/333 [00:00<?, ?it/s]100%|██████████| 333/333 [19:07<00:00,  3.44s/it]


In [ ]:
import numpy as np
final_img_features_array = np.array(final_img_features)
# Save the array
np.save(f'{repo_dir}/feature/{feature_save_name}.npy', final_img_features_array)

